<center> <h1>Robotyka 2, Zadanie 1</h1> </center>
<center> <h2>Jan Bronicki, Kacper Gąsieniec</h2> </center>

# 1. Cel zadania
Celem zadanie jest zabadanie fizycznych parametrów robota FANUC oraz jego strukturą kinematyczną.
W celu obliczenia fizycznych parametrów robota obliczamy macierz kinematyki oraz macierze transformacji pomiędzy kolejnymi układami współrzędnych z podanej tabeli Denavita-Hartenberga:

|Ogniwo | $a_i$ | $\alpha_i$       | $d_i$  | $\theta_i$            |
|:-----:|:-----:|:----------------:|:------:|---------------------|
|$1$    | $d_1$ | $-\frac{\pi}{2}$ | $0$    | $q_1$                 |
|$2$    | $d_2$ | $\pi$            | $0$    | $q_2 - \frac{\pi}{2}$ |
|$3$    | $d_3$ | $-\frac{\pi}{2}$ | $0$    | $q_2 + q_3$           |
|$4$    | $0$   | $\frac{\pi}{2}$  | $-d_4$ | $q_4$                 |
|$5$    | $0$   | $-\frac{\pi}{2}$ | $0$    | $q_5$                 |
|$6$    | $0$   | $0$              | $-d_5$ | $q_6$                 |


# 2. Obliczenia
Do oblcizeń używamy Python'a. Przeprowadzamy obliczenia symboliczne za pomocą biblioteki SymPy:

In [48]:
from sympy import *
from rotations import *
# Deklarujemy zmienne symboliczne jakie moglibysmy potrzebowac
q1, q2, q3, q4, q5, q6 = symbols('q1, q2, q3, q4, q5, q6', real=True)
a, a1, a2, a3 = symbols('a a1 a2 a3', real=True)
d, d1, d2, d3, d4, d5 = symbols('d, d1, d2, d3, d4, d5', real=True)
alfa, beta = symbols('\\alpha \\beta', real=True)
x, y, z = symbols('x y z')

# $A_{01}=$

In [49]:
A01 = rot(z, q1)*trans(z, 0)*trans(x, d1)*rot(x, -pi/2)
A01

Matrix([
[cos(q1),  0, -sin(q1), d1*cos(q1)],
[sin(q1),  0,  cos(q1), d1*sin(q1)],
[      0, -1,        0,          0],
[      0,  0,        0,          1]])

# $A_{12}=$

In [50]:
A12 = rot(z, q2-(pi/2))*trans(z, 0)*trans(x, d2)*rot(x, pi)
A12

Matrix([
[ sin(q2), -cos(q2),  0,  d2*sin(q2)],
[-cos(q2), -sin(q2),  0, -d2*cos(q2)],
[       0,        0, -1,           0],
[       0,        0,  0,           1]])

# $A_{23}=$

In [51]:
A23 = rot(z, q2+q3)*trans(z, 0)*trans(x, d3)*rot(x, -pi/2)
A23

Matrix([
[cos(q2 + q3),  0, -sin(q2 + q3), d3*cos(q2 + q3)],
[sin(q2 + q3),  0,  cos(q2 + q3), d3*sin(q2 + q3)],
[           0, -1,             0,               0],
[           0,  0,             0,               1]])

# $A_{34}=$

In [52]:
A34 = rot(z, q4)*trans(z, -d4)*trans(x, 0)*rot(x, pi/2)
A34

Matrix([
[cos(q4), 0,  sin(q4),   0],
[sin(q4), 0, -cos(q4),   0],
[      0, 1,        0, -d4],
[      0, 0,        0,   1]])

# $A_{45}=$

In [53]:
A45 = rot(z, q5)*trans(z, 0)*trans(x, 0)*rot(x, -pi/2)
A45

Matrix([
[cos(q5),  0, -sin(q5), 0],
[sin(q5),  0,  cos(q5), 0],
[      0, -1,        0, 0],
[      0,  0,        0, 1]])

# $A_{56}=$

In [54]:
A56 = rot(z, q6)*trans(z, -d5)*trans(x, 0)*rot(x, 0)
A56

Matrix([
[cos(q6), -sin(q6), 0,   0],
[sin(q6),  cos(q6), 0,   0],
[      0,        0, 1, -d5],
[      0,        0, 0,   1]])

# $A_{03}=A_{01}A_{12}A_{23}=$

In [55]:
A03 = simplify(A01*A12*A23)
A03

Matrix([
[-sin(q3)*cos(q1), -sin(q1), -cos(q1)*cos(q3), (d1 + d2*sin(q2) - d3*sin(q3))*cos(q1)],
[-sin(q1)*sin(q3),  cos(q1), -sin(q1)*cos(q3), (d1 + d2*sin(q2) - d3*sin(q3))*sin(q1)],
[         cos(q3),        0,         -sin(q3),                d2*cos(q2) + d3*cos(q3)],
[               0,        0,                0,                                      1]])

# $K=$

In [56]:
K = simplify(A12*A23*A34*A45*A56)
K

Matrix([
[-(sin(q3)*cos(q4)*cos(q5) + sin(q5)*cos(q3))*cos(q6) + sin(q3)*sin(q4)*sin(q6),  (sin(q3)*cos(q4)*cos(q5) + sin(q5)*cos(q3))*sin(q6) + sin(q3)*sin(q4)*cos(q6), sin(q3)*sin(q5)*cos(q4) - cos(q3)*cos(q5),  d2*sin(q2) - d3*sin(q3) + d4*cos(q3) - d5*sin(q3)*sin(q5)*cos(q4) + d5*cos(q3)*cos(q5)],
[ (sin(q3)*sin(q5) - cos(q3)*cos(q4)*cos(q5))*cos(q6) + sin(q4)*sin(q6)*cos(q3), (-sin(q3)*sin(q5) + cos(q3)*cos(q4)*cos(q5))*sin(q6) + sin(q4)*cos(q3)*cos(q6), sin(q3)*cos(q5) + sin(q5)*cos(q3)*cos(q4), -d2*cos(q2) - d3*cos(q3) - d4*sin(q3) - d5*sin(q3)*cos(q5) - d5*sin(q5)*cos(q3)*cos(q4)],
[                                     sin(q4)*cos(q5)*cos(q6) + sin(q6)*cos(q4),                                     -sin(q4)*sin(q6)*cos(q5) + cos(q4)*cos(q6),                          -sin(q4)*sin(q5),                                                                      d5*sin(q4)*sin(q5)],
[                                                                             0,                             

In [57]:
K_kart = K.col(3)
K_kart.row_del(3)
K_kart # Tr

Matrix([
[ d2*sin(q2) - d3*sin(q3) + d4*cos(q3) - d5*sin(q3)*sin(q5)*cos(q4) + d5*cos(q3)*cos(q5)],
[-d2*cos(q2) - d3*cos(q3) - d4*sin(q3) - d5*sin(q3)*cos(q5) - d5*sin(q5)*cos(q3)*cos(q4)],
[                                                                     d5*sin(q4)*sin(q5)],
[                                                                                      1]])

In [58]:
simplify(K_kart)

Matrix([
[ d2*sin(q2) - d3*sin(q3) + d4*cos(q3) - d5*sin(q3)*sin(q5)*cos(q4) + d5*cos(q3)*cos(q5)],
[-d2*cos(q2) - d3*cos(q3) - d4*sin(q3) - d5*sin(q3)*cos(q5) - d5*sin(q5)*cos(q3)*cos(q4)],
[                                                                     d5*sin(q4)*sin(q5)],
[                                                                                      1]])